### Description:

AES-128 algorithm with a hardcoded key is available as service.

To perform encryption, the binary requires 16-bytes plaintext (in hex):

```
    ../wbcenc --ptext="010203040506070809000a0b0c0d0e1f"
```    
The result is printed to console:
```
    3F7BA33CBAF53C4BBE5E849C3456E6DA 7A9158C14608067002F...B7ACD9201C4A5A6BB3715
```    
where:
```
    3F7BA33CBAF53C4BBE5E849C3456E6DA is a ciphertext
    
    7A9158C14608067002F...B7ACD9201C4A5A6BB3715 is a trace (information left by a programmer)
```
### Tips:

A 'trace' is a side-channel information left by a programmer to debug the implementation.

In this example a trace consists of random information plus at certain position the trace has Hamming weights of a 10th round input state bytes of the AES-128 algorithm, i.e., a State before the Sbox operation in the last round.

### Task:

Your task is to find the Master key (round key 0) embedded into the binary. 

The master key is in the form of HEIG{XXXXXXXXXX}, where X is an ASCII printable symbol.

### Leakage illustration

<img src="support/Slide.png">

### ATTACK CODE
Here you need to implement your attack which gives you the last round key.

Once the last round key is found - you need to compute the master key: this can be done with the function get_master_key()

### Programming tips
#### Numpy code
* np.arange(256).astype(np.uint8)
* np.bitwise_xor(m1, m2)
* np.where()

#### Code prepared for training
* sca_training.invSbox[sbox_out]
* sca_training.shift_rows(trace)

#### Various cod
* ''.join('{:02x}'.format(c) for c in int_array)

In [3]:
# output, ctext, trace = binary_aes128_encrypt("010203040506070809000a0b0c0d0e1f", verbose=True)
import numpy as np
import sca_training

def hex_int(hex):
    return np.array(list(bytearray.fromhex(hex)))

traces = np.array([["3F7BA33CBAF53C4BBE5E849C3456E6DA", "67BF8AF5C88AE141159BA33292D6900211BF2499854F52EA6FB17FA87D4FF6234D8921541C0B9667DD42CFAE2168E66966130B2B98931E4684D3EE0A61ED2DAEB5840CD1C5D8030303030603020503040304030303052DBAEE25035951301010EA5220B9"],
["84C13285605F717A2FE99914967051BC", "56C57D73AA1AD9E1B23DA527213256C4AB1E4F16956008134AAA10093F7B069549B91133D3291D8E6601EBBD6A80C01E9F19347379738602269641655148399A0A4A0C1DA9290304020405020406060405050504030453F615959A661CD43F5C5D4C7946"],
["0934B63731D0BDB8358316E17774C77B", "87CA656E7ECFEADF97C8C13CB97FB005248E5B00D5E76FF11196E42A08B65E8F89023C46D2302E72012EE5BAAE9EF611355A474A4AB6445B8B31BC94E8232C7A259F0000CF2E060503020503040403040304050403044798DF0C35D02F98898B4089C14E"],
["50DE2DE72C88B5564CF32D7DEC5C52E5", "1C610D075C003AB9599F3AE251F59EA4F43D5E045F66C6A998AB0A17251B8177B2C47F18C4EFD15CCD147D2748250B45629F7F000E4FA9A603EAF35E0E7D14C08093D883C2E80304030404040703040507040205050312193F0EAD83C2C18281949A0E95"],
["9EEF36FA2E33B9E9FF669B299201CA11", "A2372E8E6927DD5AE24029BC6EE7BA6032DF350A2E0C9642802CC288B2929B5DC9CA2A71F147CC1C873417F524D1958CF0CA965D156BA09698A15D893C01E70ECB50B50597C003020403030103010303020302050305243B2881C59AB8EBDE8D449DC81B"],
["518257A509D5DD5F20671B4156519138", "03BBBEDEB7608F98AB828DB81AB9C21C39DD256CD2F1C1BC464353AAEC62E1F027A9161D12A5B5F36681EBB679ECD3B3D3375EAE312873ADA10660976881C6C533DC217BC016030204040505030502040303030506028B73DA2A4981AB1850DE33719932"],
["56E45AB4D9BD511E588E274E8D05FCC0", "2C86C52416BD55C7670C75971DC1D82561BA8FDE74EA6F2B19462B9D5E9961C02965E4752C7945C9BBF06AD8F04B065A0ECC77B9F5262317A24FB409E8550850BAEDC625A54A04040104030103050206060603050205A78B9B9BCAC3F0D31CE9082118E3"],
["2D654EB67C7A26D1701441DB1952BFCD", "A2A47D06994F033F7E26461517478EC85DB90AF3F4E7D94998DD4B41E02A6F8BCEEC92A67ACB2402F1A04D47E8DB4E84D45980107F6259567FA497680D0F321A3A0300EBCE2403060506020503030302040407040604148FBEAB36CCF19F256AA2255EB0"],
["FDDE47D9FDB6C21BEC0ED917CFE54D64", "10A919B34FB695834421C688879F1132C6604BEDBCBDAACBC3E7DDBC1642A45C2ABD4E797D223C00430B89CBE1D03CB06F87DC6B838F3F4FB55C150B9EB96707B6B6813CD8F3030405050404050602050406040405030C4AC82155A51AF2ADD0B16D4BC9"],
["955F48A1AD4E3A071AB74FC6134610FD", "6A10120E8B097557C811401550D456C0A12E3E7294F3583742132A30DABEE6840E0192990B10F012583A28DE4DB4A72D21E6D6B62137ED638020C9641EB8E82CBAB9040409F502020502020405070502020604060603D7F5CAAA98276B898964818E9E8A"],
["801A6A842A4B4814FBC529BF8E443336", "1A3855D846495088D2D0E1C15A90180E02F4AB961A4E92EED77283388C9465DDCCBABE510C4D181E2603DFB7930004CB34E56A8472057B53AE3DC179D22F5FDD295C386BA98606040601020504040605040205050403F3C3CDF445DE2CDBC48B403B0029"],
["A783766D838EB6B22ADB8791BF3E855D", "4AC8706690137F59CD89C61CA9D4F0EF46BEC62E6882D705B118EE97C8976C52681BEE3765AC903B3E9557E772511EB9182326B6DB07F1951FE86B268817AE2F32A59C97906B040303050205070504030502060405042E6CE89A92AFE7801D2264B9F0F5"],
["4A770EB590903920EF29DD57345C9E93", "B01D2DE880DED41FBB9F0DA9F2A6E87D978357D528266FB388C50A09E298EA9BB556C37473A0C93748D620798611355C958C3AF3E9A9E57AADEFB998C6E273BA423F6DB5DF3F060404040405020505060402030306058B4FB68326866F997FE7D8EDDBF6"],
["A8F0AD4121CB2D787BDDA74C3CC7D140", "3499D2C22B29E1A82814687B3E2C334D34BE44D8DE7F5B217C6F95983D8AD6A72CB1738D1A933F78A8DDF3251230A87D2DED941475EF6B309D09C9DAC9DEC1F6CF3DC2E9D04002030305070005060505020306030202D55E00DD78C9034207D2B509C4C4"],
["075827586105BA6F5854F6A45FD280CE", "359E06B811DCAD74CA47B48F559C0CBF0F081779F6CF6F1CA2C9AC9082F674B7D3B179E5CC65629FAC56374031433E40828CF0B76468D4457189D5F3BE89EA9B79A2BF4E106002040403030406050202050305020404EE7804CDABC3959E9D1749656559"],])

ciphers = np.array([hex_int(trace) for trace in traces[:,0]]).transpose()
hws = np.array([hex_int(trace) for trace in traces[:,1]]).transpose().tolist()

k10 = [0]*16

for i in range(0,16):
    for k in range(256):
        cand = sca_training.HW_uint8[sca_training.invSbox[np.bitwise_xor(ciphers[i], k)]]
        if cand.tolist() in hws:
            k10[i] = k
            break

k10 = np.array(k10)
mk = sca_training.inverse_key_expansion(k10)[0][0]
flag = ''.join(chr(c) for c in mk)
flag

(16, 15)


NameError: name 'sca_training' is not defined